In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import cvxpy as cp
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# ---- STEP 1: Load and Merge Data ----

# Paths to your data files (update these paths)
hackathon_sample_v2_path = "/Users/isaiah/Desktop/Career/Clubs : Groups/Quant Hackathon/McGill-FIAM Asset Management Hackathon/ml-algorithmic-trading/Data/hackathon_sample_v2.csv"
output_path = "/Users/isaiah/Desktop/Career/Clubs : Groups/Quant Hackathon/McGill-FIAM Asset Management Hackathon/ml-algorithmic-trading/Data/full_dataset_predictions.csv"
mkt_path = "/Users/isaiah/Desktop/Career/Clubs : Groups/Quant Hackathon/McGill-FIAM Asset Management Hackathon/ml-algorithmic-trading/Data/mkt_ind.csv"

# Load data from hackathon_sample_v2.csv
hackathon_sample_v2 = pd.read_csv(
    hackathon_sample_v2_path,
    usecols=[
        'permno', 'date', 'market_equity', 'be_me', 'ret_12_1',
        'ivol_capm_21d', 'stock_exret'
    ]
)

# Load predictions from output.csv
output = pd.read_csv(output_path, usecols=['permno', 'date', 'Predicted_Excess_Return'])

# Convert 'date' columns to datetime format
hackathon_sample_v2['date'] = pd.to_datetime(hackathon_sample_v2['date'], format='%Y%m%d')  # Ensure correct date format
output['date'] = pd.to_datetime(output['date'])  # Already in datetime, but this ensures uniformity

# Merge datasets on 'permno' and 'date'
pred = pd.merge(hackathon_sample_v2, output, on=['permno', 'date'], how='inner')

# Check merged data
print(pred.head())

In [4]:
# ---- STEP 2: Data Cleaning ----

# Drop rows with any null values
pred = pred.dropna(axis=0).reset_index(drop=True)

# Ensure 'stock_exret' is numeric
pred['stock_exret'] = pd.to_numeric(pred['stock_exret'], errors='coerce')
pred = pred.dropna(subset=['stock_exret'])

# Extract 'year' and 'month' from 'date'
pred['year'] = pred['date'].dt.year
pred['month'] = pred['date'].dt.month



In [ ]:
# ---- STEP 3: Multi-Signal Ensemble with Random Forest ----

# Create additional signals
pred['value_signal'] = pred['be_me']                # Value factor
pred['momentum_signal'] = pred['ret_12_1']          # Momentum factor
pred['risk_signal'] = 1 / pred['ivol_capm_21d']     # Inverse volatility
pred['Predicted_Excess_Return_signal'] = pred['Predicted_Excess_Return']                    # ElasticNet predictions

# Create target variable
pred['stock_exret_next'] = pred.groupby('permno')['stock_exret'].shift(-1)

# Drop rows where 'stock_exret_next' is NaN (these would be the last months for each stock)
pred = pred.dropna(subset=['stock_exret_next'])

# Features and target
features = pred[['value_signal', 'momentum_signal', 'risk_signal', 'Predicted_Excess_Return_signal']]
target = pred['stock_exret_next']

# Sort data by date
pred = pred.sort_values(by='date')

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, shuffle=False
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# RandomizedSearchCV for hyperparameter tuning
rf = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    random_state=42,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

# Fit the model on the training data
random_search.fit(X_train_scaled, y_train)

# Best estimator from hyperparameter tuning
best_rf = random_search.best_estimator_

# Fit the model with the best parameters on the training data
best_rf.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred = best_rf.predict(X_test_scaled)

# Define test_data based on the test set indices
test_data = pred.loc[X_test.index].copy()

# Predict on the test data
test_data['predicted_return'] = best_rf.predict(X_test_scaled)

# Add predicted_return back to the full dataset
pred.loc[test_data.index, 'predicted_return'] = test_data['predicted_return']

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the best parameters and performance metrics
print(f"Best Parameters: {random_search.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")
print(f"Mean Absolute Error: {mae}")

# ---- Verify 'predicted_return' Exists ----
print("\nColumns in 'pred' DataFrame:", pred.columns.tolist())
print("'predicted_return' Missing Values:", pred['predicted_return'].isnull().sum())


In [ ]:
# ---- STEP 4: Monthly Rebalancing with Black-Litterman Optimization ----

# Try using on portfolio's rather then individual stocks, SML, HML, etc.

# Check that market cap is not forward looking
# pred['be_me'] = pred['be_me'].shift(-1)

# Initialize list to store monthly portfolio data
monthly_results = []

# Get list of unique months
pred['month_period'] = pred['date'].dt.to_period('M')
unique_months = pred['month_period'].unique()

for current_month in unique_months:
    # Filter data for the current month
    monthly_data = pred[pred['month_period'] == current_month].copy()
    
    if monthly_data.empty:
        continue
    
    # Calculate market weights
    monthly_data['market_weight'] = (
        monthly_data['market_equity'] / monthly_data['market_equity'].sum()
    )
    
    # Calculate market equilibrium return
    market_equilibrium_return = np.dot(
        monthly_data['market_weight'], monthly_data['stock_exret']
    )
    
    # Black-Litterman adjusted returns
    tau = 0.05  # Confidence in views
    monthly_data['bl_adjusted_returns'] = (
        (1 - tau) * market_equilibrium_return + tau * monthly_data['predicted_return']
    )
    
    # Create covariance matrix using rolling window (e.g., past 12 months)
    start_date = (current_month - 11).to_timestamp()
    end_date = current_month.to_timestamp()
    window_data = pred[
        (pred['date'] >= start_date) & (pred['date'] <= end_date)
    ]
    
    # Pivot returns for covariance calculation
    pivoted_returns = window_data.pivot(
        index='date', columns='permno', values='stock_exret'
    )
    
    # Drop assets with missing data
    pivoted_returns = pivoted_returns.dropna(axis=1, how='any')
    
    # Check if there is enough data
    if pivoted_returns.shape[0] < 2 or pivoted_returns.shape[1] == 0:
        print(f"Not enough data for covariance matrix in {current_month}")
        continue
    
    # Calculate covariance matrix
    cov_matrix_df = pivoted_returns.cov()
    cov_matrix_df = (cov_matrix_df + cov_matrix_df.T) / 2  # Ensure symmetry
    
    # Align assets in monthly_data with covariance matrix
    common_permnos = monthly_data['permno'].isin(cov_matrix_df.columns)
    monthly_data = monthly_data[common_permnos]
    
    if monthly_data.empty:
        print(f"No matching assets in {current_month}")
        continue
    
    monthly_data.set_index('permno', inplace=True)
    monthly_data = monthly_data.loc[cov_matrix_df.columns.intersection(monthly_data.index)]
    
    # Extract adjusted returns and covariance matrix
    bl_adjusted_returns = monthly_data['bl_adjusted_returns'].values
    cov_matrix = cov_matrix_df.loc[monthly_data.index, monthly_data.index].values
    
    # Ensure covariance matrix is positive semidefinite
    eigenvalues = np.linalg.eigvalsh(cov_matrix)
    if np.any(eigenvalues < 0):
        epsilon = 1e-6
        cov_matrix += epsilon * np.eye(cov_matrix.shape[0])
    
    # Wrap covariance matrix as CVXPY constant
    cov_matrix_cvx = cp.Constant(cov_matrix)
    
    # Define optimization variables
    n_assets = len(bl_adjusted_returns)
    weights = cp.Variable(n_assets)
    
    # Define objective function and constraints
    delta = 2.5  # Risk aversion parameter
    portfolio_return = bl_adjusted_returns @ weights
    portfolio_risk = cp.quad_form(weights, cov_matrix_cvx)
    objective = cp.Maximize(portfolio_return - (delta / 2) * portfolio_risk)
    constraints = [
        cp.sum(weights) == 1,
        weights >= -0.05,  # Allow short selling with a max short position of 5%
        weights <= 0.05    # Max long position is 5%
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    if problem.status not in ["infeasible", "unbounded"]:
        # Assign optimized weights
        monthly_data['optimal_weight'] = weights.value
        monthly_data.reset_index(inplace=True)
        monthly_results.append(monthly_data)
    else:
        print(f"Optimization failed for {current_month}")
        continue

# Combine monthly data
if monthly_results:
    final_portfolio = pd.concat(monthly_results, ignore_index=True)
else:
    print("No monthly results to combine.")
    final_portfolio = pd.DataFrame()

In [ ]:
# ---- STEP 5: Construct Long-Short Portfolio ----

# Select top and bottom N stocks based on optimized weights
N = 50  # Number of stocks for long and short positions
final_portfolio = final_portfolio.sort_values(
    by=['date', 'optimal_weight'], ascending=[True, False]
)

# Assign positions and weights for each month
portfolio_list = []

for date, group in final_portfolio.groupby('date'):
    group = group.copy()
    group = group.sort_values(by='optimal_weight', ascending=False)
    
    long_positions = group.head(N)
    short_positions = group.tail(N)
    
    long_positions['position'] = 'long'
    short_positions['position'] = 'short'
    
    # Assign weights
    long_positions['weight'] = long_positions['optimal_weight'] / long_positions['optimal_weight'].sum()
    short_positions['weight'] = -short_positions['optimal_weight'] / short_positions['optimal_weight'].sum()
    
    # Combine positions
    combined = pd.concat([long_positions, short_positions])
    portfolio_list.append(combined)

# Combine all positions
final_portfolio = pd.concat(portfolio_list, ignore_index=True)

In [ ]:
# ---- STEP 6: Calculate Portfolio Performance Metrics ----

# Calculate weighted returns
final_portfolio['weighted_return'] = final_portfolio['stock_exret'] * final_portfolio['weight']

# Calculate daily portfolio returns
portfolio_returns = final_portfolio.groupby('date')['weighted_return'].sum()

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns).cumprod()

# Performance metrics
# Assuming portfolio_returns is monthly
mean_return = portfolio_returns.mean()
std_return = portfolio_returns.std()

# Annualize return and volatility for the Sharpe ratio
annualized_return = mean_return * 12
annualized_volatility = std_return * np.sqrt(12)
sharpe_ratio = annualized_return / annualized_volatility
print("Sharpe Ratio:", sharpe_ratio)

annualized_return = mean_return * 12
print("Annualized Return:", annualized_return)

annualized_std_dev = std_return * np.sqrt(12)
print("Annualized Standard Deviation:", annualized_std_dev)

In [ ]:
# ---- STEP 7: Calculate CAPM Alpha and Information Ratio ----

# Load market data
mkt = pd.read_csv(mkt_path)

# Create 'mkt_rf' by subtracting risk-free rate
mkt['mkt_rf'] = mkt['sp_ret'] - mkt['rf']

# Prepare portfolio returns data (make sure 'date' exists in portfolio_returns)
portfolio_returns = portfolio_returns.reset_index()

# Create 'year' and 'month' columns in portfolio_returns (if they don't exist already)
portfolio_returns['year'] = portfolio_returns['date'].dt.year
portfolio_returns['month'] = portfolio_returns['date'].dt.month

# Merge portfolio and market data on 'year' and 'month'
regression_data = pd.merge(
    portfolio_returns, mkt[['year', 'month', 'mkt_rf']], on=['year', 'month'], how='inner'
)

# Perform CAPM regression
nw_ols = sm.ols(formula="weighted_return ~ mkt_rf", data=regression_data).fit(
    cov_type="HAC", cov_kwds={"maxlags": 3}, use_t=True
)
print(nw_ols.summary())

# Extract metrics
alpha = nw_ols.params['Intercept']
t_stat = nw_ols.tvalues['Intercept']
info_ratio = alpha / np.sqrt(nw_ols.mse_resid) * np.sqrt(12)

print("CAPM Alpha:", alpha)
print("t-statistic:", t_stat)
print("Information Ratio:", info_ratio)

In [ ]:
# ---- STEP 8: Calculate Maximum Drawdown ----

# Maximum Drawdown Calculation
if not cumulative_returns.empty:
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
else:
    max_drawdown = 0  # No drawdown in empty or zero-return periods
    
print("Maximum Drawdown:", max_drawdown)


In [ ]:
# ---- STEP 9: Calculate Turnover ----

def calculate_turnover(portfolio):
    portfolio['month'] = portfolio['date'].dt.to_period('M')
    turnover_list = []
    months = sorted(portfolio['month'].unique())

    for i in range(1, len(months)):
        current_month = months[i]
        previous_month = months[i - 1]

        current_weights = portfolio[portfolio['month'] == current_month].set_index('permno')['optimal_weight']
        previous_weights = portfolio[portfolio['month'] == previous_month].set_index('permno')['optimal_weight']
        
        # Align the two weight vectors
        all_permnos = current_weights.index.union(previous_weights.index)
        current_weights = current_weights.reindex(all_permnos, fill_value=0)
        previous_weights = previous_weights.reindex(all_permnos, fill_value=0)

        # Turnover is the sum of the absolute changes in weights
        turnover = (current_weights - previous_weights).abs().sum() / 2
        turnover_list.append(turnover)

    average_turnover = np.mean(turnover_list)
    return average_turnover

long_portfolio = final_portfolio[final_portfolio['position'] == 'long']
short_portfolio = final_portfolio[final_portfolio['position'] == 'short']

long_turnover = calculate_turnover(long_portfolio)
short_turnover = calculate_turnover(short_portfolio)
print("Long Portfolio Turnover:", long_turnover)
print("Short Portfolio Turnover:", short_turnover)
